In [13]:
#https://medium.com/@jdwittenauer/deep-learning-with-keras-recommender-systems-e7b99cb29929
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
DATA_PATH = '/datasets/book-rec/'

BOOK_RATINGS_FILE = DATA_PATH + 'BX-Book-Ratings.csv'
BOOKS_FILE = DATA_PATH + 'BX-Books.csv'
USERS_FILE = DATA_PATH + 'BX-Users.csv'

In [15]:
df_ratings = pd.read_csv(BOOK_RATINGS_FILE, delimiter=';',encoding='latin-1', error_bad_lines=False)
df_ratings = df_ratings.rename(columns={'User-ID':'userId', 'ISBN':'itemId', 'Book-Rating':'rating'})
df_ratings.head()

,userId,itemId,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [29]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

user_enc = LabelEncoder()
df_ratings['user'] = user_enc.fit_transform(df_ratings['userId'].values)
n_users = df_ratings['user'].nunique()

item_enc = LabelEncoder()
df_ratings['item'] = item_enc.fit_transform(df_ratings['itemId'].values)
n_items = df_ratings['item'].nunique()
df_ratings['rating'] = df_ratings['rating'].values.astype(np.float32)

X = df_ratings[['user', 'item']].values
y = df_ratings['rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

n_factors = 50
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

min_rating = min(df_ratings['rating'])
max_rating = max(df_ratings['rating'])

In [25]:
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Reshape, Dot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

def RecommenderV1(n_users, n_movies, n_factors):
    user = Input(shape=(1,))
    u = Embedding(n_users, n_factors, embeddings_initializer='he_normal',
                  embeddings_regularizer=l2(1e-6))(user)
    u = Reshape((n_factors,))(u)

    movie = Input(shape=(1,))
    m = Embedding(n_movies, n_factors, embeddings_initializer='he_normal',
                  embeddings_regularizer=l2(1e-6))(movie)
    m = Reshape((n_factors,))(m)

    x = Dot(axes=1)([u, m])
    model = Model(inputs=[user, movie], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

model = RecommenderV1(n_users, n_items, n_factors)
model.summary()

history = model.fit(x=X_train_array, y=y_train, batch_size=2048*16, epochs=150, verbose=1, validation_data=(X_test_array, y_test))

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_16 (Embedding)        (None, 1, 50)        5264150     input_17[0][0]                   
__________________________________________________________________________________________________
embedding_17 (Embedding)        (None, 1, 50)        17027800    input_18[0][0]                   
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [30]:
from tensorflow.keras.layers import Add, Activation, Lambda

class EmbeddingLayer:
    def __init__(self, n_items, n_factors):
        self.n_items = n_items
        self.n_factors = n_factors
    
    def __call__(self, x):
        x = Embedding(self.n_items, self.n_factors, embeddings_initializer='he_normal',
                      embeddings_regularizer=l2(1e-6))(x)
        x = Reshape((self.n_factors,))(x)
        return x

def RecommenderV2(n_users, n_movies, n_factors, min_rating, max_rating):
    user = Input(shape=(1,))
    u = EmbeddingLayer(n_users, n_factors)(user)
    ub = EmbeddingLayer(n_users, 1)(user)
    
    movie = Input(shape=(1,))
    m = EmbeddingLayer(n_movies, n_factors)(movie)
    mb = EmbeddingLayer(n_movies, 1)(movie)
    x = Dot(axes=1)([u, m])
    x = Add()([x, ub, mb])
    x = Activation('sigmoid')(x)
    x = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(x)
    model = Model(inputs=[user, movie], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

model = RecommenderV2(n_users, n_items, n_factors, min_rating, max_rating)
model.summary()
history = model.fit(x=X_train_array, y=y_train, batch_size=2048*16, epochs=150, verbose=1, validation_data=(X_test_array, y_test))

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, 1, 50)        5264150     input_19[0][0]                   
__________________________________________________________________________________________________
embedding_20 (Embedding)        (None, 1, 50)        17027800    input_20[0][0]                   
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
